# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_weather_path = "../WeatherPy/weather_search.csv"
weather_df = pd.read_csv(csv_weather_path)

In [3]:
weather_df

,city,city_id,country,latitude,longitude,temperature,max_temp,cloudiness,humidity,wind_speed
0,São Gabriel da Cachoeira,3662342,BR,-0.1303,-67.0892,73.67,73.67,95,97,0.78
1,Portland,5746545,US,45.5234,-122.6762,49.84,51.01,100,84,3.00
2,Sovetskiy,1491230,RU,61.3614,63.5842,44.15,44.15,100,66,9.10
3,Albany,5106841,US,42.6001,-73.9662,50.31,53.01,100,45,1.83
4,Fairbanks,5861897,US,64.8378,-147.7164,61.20,62.60,1,23,3.44
...,...,...,...,...,...,...,...,...,...,...
556,Uvarovo,477626,RU,51.9849,42.2615,36.00,36.00,29,77,16.53
557,Chingirlau,610091,KZ,51.0950,54.0814,48.97,48.97,77,77,17.00
558,Avallon,3035768,FR,47.4833,3.9000,46.00,46.00,72,67,6.55
559,Medininagar,1273626,IN,24.0333,84.0667,97.92,97.92,1,9,6.26


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

locations = weather_df[["latitude","longitude"]].astype(float)
humidity=weather_df["humidity"].astype(float)

figure_layout = {
    'width': '500px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,map_type='ROADMAP')

heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
vacay_options=weather_df[(weather_df['max_temp']>70)&(weather_df['max_temp']<80)&(weather_df['humidity']<30)&(weather_df['cloudiness']<30)]
vacay_options.head()

,city,city_id,country,latitude,longitude,temperature,max_temp,cloudiness,humidity,wind_speed
48,Cabo San Lucas,3985710,MX,22.8909,-109.9124,71.53,72.00,1,21,10.22
70,Saint George,5546220,US,37.1041,-113.5841,74.08,75.20,1,15,14.97
140,Adrar,2381972,MR,20.5022,-10.0711,75.15,75.15,6,16,8.48
317,Alice Springs,2077895,AU,-23.7000,133.8833,73.72,73.99,0,25,10.36
391,Yulara,6355222,AU,-25.2406,130.9889,73.40,73.40,0,27,16.11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
hotel_df=pd.DataFrame(vacay_options,columns=["city","country","hotel_lat","hotel_lng"])
hotel_df["hotel_name"]=""
#hotel_df['hotel_lat']=""
#hotel_df["hotel_lon"]=""

hotel_df


,city,country,hotel_lat,hotel_lng,hotel_name
48,Cabo San Lucas,MX,NaN,NaN,
70,Saint George,US,NaN,NaN,
140,Adrar,MR,NaN,NaN,
317,Alice Springs,AU,NaN,NaN,
391,Yulara,AU,NaN,NaN,
462,Mīrābād,AF,NaN,NaN,
469,Birin,NP,NaN,NaN,
515,Khāsh,IR,NaN,NaN,


In [41]:
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#location="22.8909,-109.9124"
#target_radius = 8000
#target_type = "hotel"
#key= g_key

#target_url = f"{base_url}&location={location}&radius={target_radius}&type={target_type}&key={g_key}"
#response = requests.get(target_url).json()
#print(json.dumps(response,indent=4,sort_keys=True))


In [42]:
hotel_name = []

#city = hotel_df['city']
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
    "radius": 8000,
    "type": "hotel",
    "key": g_key
}

for index, row in vacay_options.iterrows():
    #get latitude & longitude
    lat=row['latitude']
    lon=row['longitude']
    city=row['city']
    #print lat & long for iteration
    params["location"]=f"{lat},{lon}"
            
    response = requests.get(base_url, params=params).json()
    
    try:
        print(f"For {city} the closest hotel is {response['results'][1]['name']}.")
        hotel_df.loc[index, 'hotel_name'] = response['results'][1]['name']
        hotel_df.loc[index, 'hotel_lat'] = response['results'][1]['geometry']['location']['lat']
        hotel_df.loc[index, 'hotel_lng'] = response['results'][1]['geometry']['location']['lng']
    except IndexError:
        print(f"No hotel avaialable in {city}")
    
    print("---------------------")


For Cabo San Lucas the closest hotel is Hotel Tesoro Los Cabos.
---------------------
For Saint George the closest hotel is Best Western Plus Abbey Inn.
---------------------
No hotel avaialable in Adrar
---------------------
For Alice Springs the closest hotel is Desert Palms Alice Springs.
---------------------
For Yulara the closest hotel is Desert Gardens Hotel - Ayers Rock Resort.
---------------------
For Mīrābād the closest hotel is Deh-e Kadila.
---------------------
For Birin the closest hotel is Rastriya Banijya Bank.
---------------------
For Khāsh the closest hotel is Badran Gostaran Co..
---------------------


In [46]:
hotel_df.columns = ['City', 'Country', 'Lat','Lng','Hotel Name']
hotel_df = hotel_df.dropna()
hotel_df


,City,Country,Lat,Lng,Hotel Name
48,Cabo San Lucas,MX,22.880938,-109.911797,Hotel Tesoro Los Cabos
70,Saint George,US,37.089025,-113.584267,Best Western Plus Abbey Inn
317,Alice Springs,AU,-23.713395,133.879820,Desert Palms Alice Springs
391,Yulara,AU,-25.243725,130.984807,Desert Gardens Hotel - Ayers Rock Resort
462,Mīrābād,AF,30.475269,61.797039,Deh-e Kadila
469,Birin,NP,26.895244,88.069087,Rastriya Banijya Bank
515,Khāsh,IR,28.218871,61.212955,Badran Gostaran Co.


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [53]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(marker_layer)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))